In [ ]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.model_selection import train_test_split
import glob
import os

In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
import os
import glob

# Initialize MobileNetV3 model
model = MobileNetV3Small(weights='imagenet', include_top=False, pooling='avg')

def extract_features(video_path):
    cap = cv2.VideoCapture(video_path)
    frames, features = [], []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        resized_frame = cv2.resize(frame, (224, 224))
        img_data = preprocess_input(np.expand_dims(resized_frame, axis=0))
        features.append(model.predict(img_data).flatten())
    cap.release()
    return frames, np.array(features)

def apply_attention_filter(features):
    attention_mask = np.zeros_like(features)
    for i, feature in enumerate(features):
        action_relevance = np.var(feature)  # Using variance for relevance
        attention_mask[i] = feature * action_relevance
    return attention_mask

def k_medoids_clustering(features, K=5):
    kmedoids = KMeans(n_clusters=K, random_state=0)
    kmedoids.fit(features)
    return kmedoids.labels_, kmedoids.cluster_centers_

def temporal_consistency(keyframe_indices, total_frames, min_interval=10, action_speed="fast"):
    if action_speed == "fast":
        min_interval = 5
    elif action_speed == "slow":
        min_interval = 15
    consistent_indices = []
    last_frame = -min_interval
    for idx in sorted(keyframe_indices):
        if idx - last_frame >= min_interval:
            consistent_indices.append(idx)
            last_frame = idx
    return consistent_indices

def determine_k_by_interval(video_path, interval=0.375, min_keyframes=3, max_keyframes=50, action_density="high"):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_per_interval = int(fps * interval)
    K = total_frames // frames_per_interval
    if action_density == "high":
        K = min(K + 10, max_keyframes)
    elif action_density == "low":
        K = max(K - 10, min_keyframes)
    cap.release()
    return K

def extract_keyframes_with_attention_and_temporal(video_path, min_interval=10, action_speed="fast", action_density="high"):
    frames, features = extract_features(video_path)
    features_with_attention = apply_attention_filter(features)
    K = determine_k_by_interval(video_path, interval=0.375, action_density=action_density)
    labels, cluster_centers = k_medoids_clustering(features_with_attention, K=K)
    keyframe_indices = []
    for center in cluster_centers:
        center_distances = np.linalg.norm(features_with_attention - center, axis=1)
        keyframe_idx = np.argmin(center_distances)
        keyframe_indices.append(keyframe_idx)
    consistent_indices = temporal_consistency(keyframe_indices, len(frames), min_interval=min_interval, action_speed=action_speed)
    keyframes = [frames[idx] for idx in consistent_indices]
    return keyframes

def process_videos(data_dir, output_dir, train_ratio=0.8):
    video_paths = glob.glob(os.path.join(data_dir, "*.mp4"))
    train_paths, test_paths = train_test_split(video_paths, test_size=1 - train_ratio, random_state=42)
    
    # Directories for train and test keyframes
    train_output_dir = os.path.join(output_dir, "train_keyframes")
    test_output_dir = os.path.join(output_dir, "test_keyframes")
    os.makedirs(train_output_dir, exist_ok=True)
    os.makedirs(test_output_dir, exist_ok=True)
    
    # Process and save train keyframes
    for i, video_path in enumerate(train_paths):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        video_output_dir = os.path.join(train_output_dir, video_name)
        os.makedirs(video_output_dir, exist_ok=True)
        
        keyframes = extract_keyframes_with_attention_and_temporal(video_path)
        for j, keyframe in enumerate(keyframes):
            keyframe_path = os.path.join(video_output_dir, f"keyframe{j+1}.jpg")
            cv2.imwrite(keyframe_path, keyframe)
            print(f"Saved {keyframe_path}")
    
    # Process and save test keyframes
    for i, video_path in enumerate(test_paths):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        video_output_dir = os.path.join(test_output_dir, video_name)
        os.makedirs(video_output_dir, exist_ok=True)
        
        keyframes = extract_keyframes_with_attention_and_temporal(video_path)
        for j, keyframe in enumerate(keyframes):
            keyframe_path = os.path.join(video_output_dir, f"keyframe{j+1}.jpg")
            cv2.imwrite(keyframe_path, keyframe)
            print(f"Saved {keyframe_path}")

# Define the path to your videos folder and output directory
data_dir = r"D:\my part"
output_dir = r"D:\virtual\keyframes"

# Process videos
process_videos(data_dir, output_dir, train_ratio=0.8)

cv2.destroyAllWindows()
